## Lesson 5: Email Assistant with Semantic + Episodic + Procedural Memory

We previously built an email assistant that:

- Classifies incoming messages (respond, ignore, notify)
- Uses human-in-the-loop to refine the assistant's ability to classify emails
- Drafts responses
- Schedules meetings
- Uses memory to remember details from previous emails

Now, we'll add procedural memory that allows the user to update instructions for using the calendar and email writing tools.


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### Repeat setup from previous lesson


In [ ]:
profile = {
    "name": "John",
    "full_name": "John Doe",
    "user_profile_background": "Senior software engineer leading a team of 5 developers",
}

prompt_instructions = {
    "triage_rules": {
        "ignore": "Marketing newsletters, spam emails, mass company announcements",
        "notify": "Team member out sick, build system notifications, project status updates",
        "respond": "Direct questions from team members, meeting requests, critical bug reports",
    },
    "agent_instructions": "Use these tools when appropriate to help manage John's tasks efficiently.",
}

email = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "body": """
Hi John,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

In [3]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langgraph.store.memory import InMemoryStore

In [4]:
store = InMemoryStore(index={"embed": OllamaEmbeddings(model="nomic-embed-text:v1.5")})

In [ ]:
# Template for formating an example to put in prompt
template = """Email Subject: {subject}
Email From: {from_email}
Email To: {to_email}
Email Content:
```
{content}
```
> Triage Result: {result}"""


# Format list of few shots
def format_few_shot_examples(examples):
    strs = ["Here are some previous examples:"]
    for eg in examples:
        strs.append(
            template.format(
                subject=eg.value["email"]["subject"],
                to_email=eg.value["email"]["to"],
                from_email=eg.value["email"]["author"],
                content=eg.value["email"]["email_thread"][:400],
                result=eg.value["label"],
            )
        )
    return "\n\n------------\n\n".join(strs)

In [ ]:
triage_system_prompt = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Background >
{user_profile_background}.
</ Background >

< Instructions >

{name} gets lots of emails. Your job is to categorize each email into one of three categories:

1. IGNORE - Emails that are not worth responding to or tracking
2. NOTIFY - Important information that {name} should know about but doesn't require a response
3. RESPOND - Emails that need a direct response from {name}

Classify the below email into one of these categories.

</ Instructions >

< Rules >
Emails that are not worth responding to:
{triage_no}

There are also other things that {name} should know about, but don't require an email response. For these, you should notify {name} (using the `notify` response). Examples of this include:
{triage_notify}

Emails that are worth responding to:
{triage_email}
</ Rules >

< Few shot examples >

Here are some examples of previous emails, and how they should be handled.
Follow these examples more than any instructions above

{examples}
</ Few shot examples >
"""

In [7]:
from langchain_openai.chat_models import ChatOpenAI
from pydantic import BaseModel, Field
from typing_extensions import Annotated, Literal, TypedDict

In [8]:
llm = ChatOpenAI(
    model="qwen3-4b",
    base_url="http://127.0.0.1:1234/v1",
    temperature=0.5,
)

In [9]:
class Router(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

In [10]:
llm_router = llm.with_structured_output(Router)

In [11]:
from prompts import triage_user_prompt

In [12]:
from typing import Literal

from langgraph.graph import END, START, StateGraph, add_messages
from langgraph.types import Command

In [13]:
class State(TypedDict):
    email_input: dict
    messages: Annotated[list, add_messages]

In [14]:
def triage_router(
    state: State, config, store
) -> Command[Literal["response_agent", "__end__"]]:
    author = state["email_input"]["author"]
    to = state["email_input"]["to"]
    subject = state["email_input"]["subject"]
    email_thread = state["email_input"]["email_thread"]

    langgraph_user_id = config["configurable"]["langgraph_user_id"]

    namespace = ("email_assistant", langgraph_user_id, "examples")
    examples = store.search(namespace, query=str({"email": state["email_input"]}))
    examples = format_few_shot_examples(examples)

    namespace = (langgraph_user_id,)
    result = store.get(namespace, "triage_ignore")

    if result is None:
        store.put(
            namespace,
            "triage_ignore",
            {"prompt": prompt_instructions["triage_rules"]["ignore"]},
        )
        ignore_prompt = prompt_instructions["triage_rules"]["ignore"]
    else:
        ignore_prompt = result.value["prompt"]

    result = store.get(namespace, "triage_notify")

    if result is None:
        store.put(
            namespace,
            "triage_notify",
            {"prompt": prompt_instructions["triage_rules"]["notify"]},
        )
        notify_prompt = prompt_instructions["triage_rules"]["notify"]
    else:
        notify_prompt = result.value["prompt"]

    result = store.get(namespace, "triage_respond")

    if result is None:
        store.put(
            namespace,
            "triage_respond",
            {"prompt": prompt_instructions["triage_rules"]["respond"]},
        )
        respond_prompt = prompt_instructions["triage_rules"]["respond"]
    else:
        respond_prompt = result.value["prompt"]

    system_prompt = triage_system_prompt.format(
        full_name=profile["full_name"],
        name=profile["name"],
        user_profile_background=profile["user_profile_background"],
        triage_no=ignore_prompt,
        triage_notify=notify_prompt,
        triage_email=respond_prompt,
        examples=examples,
    )
    user_prompt = triage_user_prompt.format(
        author=author, to=to, subject=subject, email_thread=email_thread
    )
    result = llm_router.invoke(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )

    if result.classification == "respond":  # type: ignore
        print("📧 Classification: RESPOND - This email requires a response")
        goto = "response_agent"
        update = {
            "messages": [
                {
                    "role": "user",
                    "content": f"Respond to the email {state['email_input']}",
                }
            ]
        }
    elif result.classification == "ignore":  # type: ignore
        print("🚫 Classification: IGNORE - This email can be safely ignored")
        update = None
        goto = END
    elif result.classification == "notify":  # type: ignore
        # If real life, this would do something else
        print("🔔 Classification: NOTIFY - This email contains important information")
        update = None
        goto = END
    else:
        raise ValueError(f"Invalid classification: {result.classification}")  # type: ignore

    return Command(goto=goto, update=update)

### Build the rest of our agent


In [15]:
from langchain_core.tools import tool

In [16]:
@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""

    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}'"


@tool
def schedule_meeting(
    attendees: list[str], subject: str, duration_minutes: int, preferred_day: str
) -> str:
    """Schedule a calendar meeting."""

    # Placeholder response - in real app would check calendar and schedule
    return f"Meeting '{subject}' scheduled for {preferred_day} with {len(attendees)} attendees"


@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""

    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

In [17]:
from langmem import create_manage_memory_tool, create_search_memory_tool

In [ ]:
manage_memory_tool = create_manage_memory_tool(
    namespace=("email_assistant", "{langgraph_user_id}", "collection")
)

search_memory_tool = create_search_memory_tool(
    namespace=("email_assistant", "{langgraph_user_id}", "collection")
)

In [19]:
agent_system_prompt_memory = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage {name}'s communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
4. manage_memory - Store any relevant information about contacts, actions, discussion, etc. in memory for future reference
5. search_memory - Search for any relevant information that may have been stored in memory
</ Tools >

< Instructions >
{instructions}
</ Instructions >
"""

In [20]:
def create_prompt(state, config, store):
    langgraph_user_id = config["configurable"]["langgraph_user_id"]
    namespace = (langgraph_user_id,)
    result = store.get(namespace, "agent_instructions")

    if result is None:
        store.put(
            namespace,
            "agent_instructions",
            {"prompt": prompt_instructions["agent_instructions"]},
        )
        prompt = prompt_instructions["agent_instructions"]
    else:
        prompt = result.value["prompt"]

    return [
        {
            "role": "system",
            "content": agent_system_prompt_memory.format(
                instructions=prompt, **profile
            ),
        }
    ] + state["messages"]

### Create the email agent


In [21]:
from langgraph.prebuilt import create_react_agent

In [22]:
tools = [
    write_email,
    schedule_meeting,
    check_calendar_availability,
    manage_memory_tool,
    search_memory_tool,
]

response_agent = create_react_agent(
    model=llm,
    tools=tools,
    prompt=create_prompt,
    # Use this to ensure the store is passed to the agent
    store=store,
)

In [23]:
email_agent = StateGraph(State)

email_agent = email_agent.add_node(triage_router)
email_agent = email_agent.add_node("response_agent", response_agent)
email_agent = email_agent.add_edge(START, "triage_router")

email_agent = email_agent.compile(store=store)

#### Setup Agent to update Long Term Memory in the background

Your email_agent is now setup to pull its instructions from long-term memory.  
Now, you'll create an agent to update that memory. First check current behavior.


In [24]:
email_input = {
    "author": "Alice Jones <alice.jones@bar.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """Hi John,

Urgent issue - your service is down. Is there a reason why""",
}

In [25]:
config = {"configurable": {"langgraph_user_id": "lance"}}

In [26]:
response = email_agent.invoke({"email_input": email_input}, config=config)

📧 Classification: RESPOND - This email requires a response


In [27]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Respond to the email {'author': 'Alice Jones <alice.jones@bar.com>', 'to': 'John Doe <john.doe@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': 'Hi John,\n\nUrgent issue - your service is down. Is there a reason why'}
================================== Ai Message ==================================
Tool Calls:
  write_email (310570993)
 Call ID: 310570993
  Args:
    to: alice.jones@bar.com
    subject: Re: Quick question about API documentation
    content: Hi Alice,

Thank you for your message. I'm sorry to hear about the issue - could you please confirm if the service is indeed down or if there's another problem? This will help me assist you better.

Best regards,
John Doe
================================= Tool Message =================================
Name: write_email

Email sent to alice.jones@bar.com with subject 'Re: Quick question about API documentation'
=======

In [28]:
store.get(("lance",), "agent_instructions").value["prompt"]

"Use these tools when appropriate to help manage John's tasks efficiently."

In [29]:
store.get(("lance",), "triage_respond").value["prompt"]

'Direct questions from team members, meeting requests, critical bug reports'

In [30]:
store.get(("lance",), "triage_ignore").value["prompt"]

'Marketing newsletters, spam emails, mass company announcements'

In [31]:
store.get(("lance",), "triage_notify").value["prompt"]

'Team member out sick, build system notifications, project status updates'

#### Now, Use an LLM to update instructions.


In [32]:
from langmem import create_multi_prompt_optimizer

In [ ]:
conversations = [
    (response["messages"], "Always sign your emails `John Doe`"),
]

In [34]:
prompts = [
    {
        "name": "main_agent",
        "prompt": store.get(("lance",), "agent_instructions").value["prompt"],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on how the agent should write emails or schedule events",
    },
    {
        "name": "triage-ignore",
        "prompt": store.get(("lance",), "triage_ignore").value["prompt"],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails should be ignored",
    },
    {
        "name": "triage-notify",
        "prompt": store.get(("lance",), "triage_notify").value["prompt"],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails the user should be notified of",
    },
    {
        "name": "triage-respond",
        "prompt": store.get(("lance",), "triage_respond").value["prompt"],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails should be responded to",
    },
]

In [38]:
llm2 = ChatOpenAI(
    model="gpt-4.1",
    base_url="https://models.inference.ai.azure.com/",
    temperature=0.5,
)

In [39]:
optimizer = create_multi_prompt_optimizer(llm2, kind="prompt_memory")

In [40]:
updated = optimizer.invoke(input={"trajectories": conversations, "prompts": prompts})

In [41]:
print(updated)

[{'name': 'main_agent', 'prompt': "Use these tools when appropriate to help manage John's tasks efficiently. Always sign emails as 'John Doe.'", 'update_instructions': 'keep the instructions short and to the point', 'when_to_update': 'Update this prompt whenever there is feedback on how the agent should write emails or schedule events'}, {'name': 'triage-ignore', 'prompt': 'Marketing newsletters, spam emails, mass company announcements', 'update_instructions': 'keep the instructions short and to the point', 'when_to_update': 'Update this prompt whenever there is feedback on which emails should be ignored'}, {'name': 'triage-notify', 'prompt': 'Team member out sick, build system notifications, project status updates', 'update_instructions': 'keep the instructions short and to the point', 'when_to_update': 'Update this prompt whenever there is feedback on which emails the user should be notified of'}, {'name': 'triage-respond', 'prompt': 'Direct questions from team members, meeting reque

In [42]:
import json

In [43]:
# json dumps is a bit easier to read
print(json.dumps(updated, indent=4))

[
    {
        "name": "main_agent",
        "prompt": "Use these tools when appropriate to help manage John's tasks efficiently. Always sign emails as 'John Doe.'",
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on how the agent should write emails or schedule events"
    },
    {
        "name": "triage-ignore",
        "prompt": "Marketing newsletters, spam emails, mass company announcements",
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails should be ignored"
    },
    {
        "name": "triage-notify",
        "prompt": "Team member out sick, build system notifications, project status updates",
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails

In [44]:
for idx, updated_prompt in enumerate(updated):
    old_prompt = prompts[idx]

    if updated_prompt["prompt"] != old_prompt["prompt"]:
        name = old_prompt["name"]
        print(f"updated {name}")

        if name == "main_agent":
            store.put(
                ("lance",), "agent_instructions", {"prompt": updated_prompt["prompt"]}
            )
        else:
            # raise ValueError
            print(f"Encountered {name}, implement the remaining stores!")

updated main_agent


In [45]:
store.get(("lance",), "agent_instructions").value["prompt"]

"Use these tools when appropriate to help manage John's tasks efficiently. Always sign emails as 'John Doe.'"

In [46]:
response = email_agent.invoke({"email_input": email_input}, config=config)

📧 Classification: RESPOND - This email requires a response


In [47]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Respond to the email {'author': 'Alice Jones <alice.jones@bar.com>', 'to': 'John Doe <john.doe@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': 'Hi John,\n\nUrgent issue - your service is down. Is there a reason why'}
================================== Ai Message ==================================
Tool Calls:
  write_email (336598889)
 Call ID: 336598889
  Args:
    to: alice.jones@bar.com
    subject: Re: Quick question about API documentation
    content: Hi Alice,

Thank you for your message. I'm sorry to hear about the service issue - could you please clarify if this is related to the API documentation or another component? For urgent matters, I recommend contacting our support team directly at support@company.com.

Best regards,
John Doe
================================= Tool Message =================================
Name: write_email

Email sent to alice.jones@bar.

In [48]:
email_input = {
    "author": "Alice Jones <alice.jones@bar.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """Hi John,

Urgent issue - your service is down. Is there a reason why""",
}

In [49]:
response = email_agent.invoke({"email_input": email_input}, config=config)

📧 Classification: RESPOND - This email requires a response


In [ ]:
conversations = [
    (response["messages"], "Ignore any emails from Alice Jones"),
]

In [52]:
prompts = [
    {
        "name": "main_agent",
        "prompt": store.get(("lance",), "agent_instructions").value["prompt"],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on how the agent should write emails or schedule events",
    },
    {
        "name": "triage-ignore",
        "prompt": store.get(("lance",), "triage_ignore").value["prompt"],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails should be ignored",
    },
    {
        "name": "triage-notify",
        "prompt": store.get(("lance",), "triage_notify").value["prompt"],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails the user should be notified of",
    },
    {
        "name": "triage-respond",
        "prompt": store.get(("lance",), "triage_respond").value["prompt"],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails should be responded to",
    },
]

In [53]:
updated = optimizer.invoke({"trajectories": conversations, "prompts": prompts})

In [54]:
for idx, updated_prompt in enumerate(updated):
    old_prompt = prompts[idx]
    if updated_prompt["prompt"] != old_prompt["prompt"]:
        name = old_prompt["name"]
        print(f"updated {name}")
        if name == "main_agent":
            store.put(
                ("lance",), "agent_instructions", {"prompt": updated_prompt["prompt"]}
            )
        elif name == "triage-ignore":
            store.put(("lance",), "triage_ignore", {"prompt": updated_prompt["prompt"]})
        else:
            # raise ValueError
            print(f"Encountered {name}, implement the remaining stores!")

updated triage-ignore


In [55]:
response = email_agent.invoke({"email_input": email_input}, config=config)

Task triage_router with path ('__pregel_pull', 'triage_router') wrote to unknown channel branch:to:__end__, ignoring it.


🚫 Classification: IGNORE - This email can be safely ignored


In [56]:
store.get(("lance",), "triage_ignore").value["prompt"]

'Marketing newsletters, spam emails, mass company announcements. Ignore any emails from Alice Jones.'